<a href="https://colab.research.google.com/github/The-DarkBeast/Humanless-Parking/blob/darknet-base/DarkNet-setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/AlexeyAB/darknet 

fatal: destination path 'darknet' already exists and is not an empty directory.


In [0]:
!cd darknet && ls

3rdparty      cmake		      image_yolov2.sh	     net_cam_v3.sh
appveyor.yml  CMakeLists.txt	      image_yolov3.sh	     README.md
build	      DarknetConfig.cmake.in  include		     scripts
build.ps1     darknet.py	      json_mjpeg_streams.sh  src
build.sh      darknet_video.py	      LICENSE		     video_v2.sh
cfg	      data		      Makefile		     video_yolov3.sh


In [0]:
!cd darknet && sed -i "s|GPU=0|GPU=1|g" Makefile

In [0]:
!ls

darknet  sample_data


In [0]:
!cd darknet && cat Makefile

GPU=1
CUDNN=0
CUDNN_HALF=0
OPENCV=0
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	  -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp, Tesla

In [0]:
!cd darknet && make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:247:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
gcc -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘convolution_2d’:
./src/gemm.c:2039:15: warning: unused variable ‘ou

In [0]:
!ls

darknet  sample_data


In [0]:
!ls darknet

3rdparty      cmake		      image_yolov2.sh	     obj
appveyor.yml  CMakeLists.txt	      image_yolov3.sh	     README.md
backup	      darknet		      include		     results
build	      DarknetConfig.cmake.in  json_mjpeg_streams.sh  scripts
build.ps1     darknet.py	      LICENSE		     src
build.sh      darknet_video.py	      Makefile		     video_v2.sh
cfg	      data		      net_cam_v3.sh	     video_yolov3.sh


In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2020-01-17 06:11:17--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  38.5MB/s    in 6.6s    

2020-01-17 06:11:24 (35.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

/bin/bash: ./darknet: Is a directory


In [0]:
mv yolov3* darknet/

In [0]:
!cd darknet && ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

 CUDA-version: 10000 (10010)
, GPU count: 1  
 OpenCV isn't used 
net.optimized_memory = 0 
batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv   

In [0]:
!ls

darknet  sample_data


In [0]:
!cd darknet && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/yolo-obj.cfg && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/yolo-obj_final_vehicle.weights

--2020-01-17 07:15:42--  https://github.com/The-DarkBeast/Humanless-Parking/raw/master/yolo-obj.cfg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/The-DarkBeast/Humanless-Parking/master/yolo-obj.cfg [following]
--2020-01-17 07:15:42--  https://media.githubusercontent.com/media/The-DarkBeast/Humanless-Parking/master/yolo-obj.cfg
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8329 (8.1K) [application/octet-stream]
Saving to: ‘yolo-obj.cfg’

yolo-obj.cfg        100%[===================>]   8.13K  --.-KB/s    in 0s      

2020-01-17 07:15:42 (118 MB/s) - ‘yolo-obj.cfg’ saved [8329/8329]

In [0]:
!ls darknet/data

9k.tree     eagle.jpg	 imagenet.labels.list	   person.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  scream.jpg
coco.names  goal.txt	 labels			   voc.names
dog.jpg     horses.jpg	 openimages.names


In [0]:
!cd darknet/data && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/obj.data && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/obj.names && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/train.txt && wget https://github.com/The-DarkBeast/Humanless-Parking/raw/master/valid.txt

--2020-01-17 07:20:22--  https://github.com/The-DarkBeast/Humanless-Parking/raw/master/obj.data
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/The-DarkBeast/Humanless-Parking/master/obj.data [following]
--2020-01-17 07:20:22--  https://media.githubusercontent.com/media/The-DarkBeast/Humanless-Parking/master/obj.data
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100 [application/octet-stream]
Saving to: ‘obj.data’

obj.data            100%[===================>]     100  --.-KB/s    in 0s      

2020-01-17 07:20:23 (16.6 MB/s) - ‘obj.data’ saved [100/100]

--2020-01-17 07:20:23--  ht